In [3]:
import pandas as pd
import os
import glob
from konlpy.tag import Okt, Kkma, Komoran
from collections import Counter

In [4]:
def createDirectory(directory): 
    """ 새로운 폴더를 생성하는 함수 """
    try: 
        if not os.path.exists(directory): 
            os.makedirs(directory) 
    except OSError: 
        print("Error: Failed to create the directory.")

In [5]:
def analysis_noun_person(df, stopwords):
    """ (의원별) 명사를 추출하여 각 명사별 개수를 추출해주는 함수. """
    okt = Okt()
    dict_person = {}
    d = {}

    for i in range(len(df['발언내용'])):        
        person = df.iloc[i, 7]
        line = df.iloc[i, 9]
    
        nouns = okt.nouns(line)    
        
        """ 한글자 단어, 불용어는 제외하고 저장함. """
        nouns = [j for j in nouns if len(j) >= 2 and j not in stopwords]
        
        if person not in dict_person:
            dict_person[person] = nouns
        else:
            dict_person[person] += nouns
        
    for j in dict_person:        
        d[j] = Counter(dict_person[j])
    
    return d

In [6]:
def make_df_person(c, p_name):
    """ 정해진 포맷에 맞게 데이터프레임을 생성해주는 함수. (의원 추가) """
    
    d = pd.DataFrame.from_dict(c, orient='index').reset_index()
    
    ''' 발언한 적은 있지만 명사에 대한 결과가 없는 경우 단어가 잡히지 않음.
    이 경우 빈 데이터프레임으로 나와 빈 데이터프레임을 반환하는 것으로 예외처리함. 
    ex) 전체 회의에서 '네, 알겠습니다.' 만 대답한 경우 '''
    try:
        d.columns = ['단어', '빈도수']
    except ValueError:
        # 빈 데이터프레임 반환
        return pd.DataFrame()
    
    count_column = df['회수']
    d['의원명'] = p_name
    d = pd.concat([d, count_column], axis = 1).dropna()
    d = d[['회수', '의원명', '단어', '빈도수']]
    
    return d  

In [18]:
location = os.getcwd()
"""
os.listdir -> 폴더 내 파일의 이름만을 가져오기 위함.
glob.glob ->  조건에 맞는 파일명을 경로를 포함하여 가져오기 위함.
ex) glob.glob('dir/*.txt') -> 모든 txt파일 가져옴.
* -> 임의 길이의 모든 문자열.
"""
fname_list = os.listdir(os.path.join(location, "회의록_모음"))
file_list = glob.glob(os.path.join(location, "회의록_모음/*"))

# 원하는 폴더 이름 설정.
dir_name = "위원 별 단어 빈도 분석"
createDirectory(dir_name)
dir_high = os.path.join(location, dir_name)

s = pd.read_csv("불용어사전_통합본.csv", encoding = 'euc-kr')
stopwords_1_2 = set(s['불용어'])


for i in range(len(fname_list)):
    num = fname_list[i][:-4]
    dir_low = os.path.join(dir_high, num)
    createDirectory(dir_low)
    
    df = pd.read_csv(file_list[i], encoding = 'euc-kr')    
    result_n_p = analysis_noun_person(df, stopwords_1_2)
    # 빈 데이터프레임이 출력되는(유의미한 명사가 회의에서 없었음.) 경우 확인.
    print([i for i in result_n_p if len(result_n_p[i]) < 1])
    keys = list(result_n_p.keys())
    
    for n in range(len(keys)):
        name = keys[n]      
        make_df_person(result_n_p[name], name).to_csv(
            os.path.join(dir_low, "%s.csv" %  name), index = False, encoding = 'euc-kr')
    print("%s 완료" % num)

서울특별시의회 회의록 제277회[정례회] (2017.11.01 ~ 2017.12.20) 완료
서울특별시의회 회의록 제284회[정례회] (2018.11.01 ~ 2018.12.20) 완료
서울특별시의회 회의록 제290회[정례회] (2019.11.01 ~ 2019.12.20) 완료
서울특별시의회 회의록 제298회[정례회](2020.11.02 ~ 2020.12.22) 완료
서울특별시의회 회의록 제303회[정례회](2021.11.01 ~ 2021.12.27) 완료
